In [1]:
#%run /Common/config_All_Zones

In [2]:
from pyspark.sql.functions import year, month, dayofmonth, hour, col
from datetime import date, timedelta 
startDate = date.today() - timedelta(days=30)

In [3]:
from pyspark.sql.functions import year, month, dayofmonth, hour, col
from datetime import date, timedelta 

startSilverDate = date.today();
  
try: startDate
except NameError: 
  print("startDate not defined setting to 30 days in past")
  startSilverDate = date.today() - timedelta(days=30)
else: 
    startSilverDate = startDate

print("Pulling data from silver starting at ", startSilverDate)

Pulling data from silver starting at 2020-06-16

In [4]:
kronosLogsAll = spark.read.format('delta').load("/mnt/silver/CAB/Ops/KronosLogs/")
kronosLogsThisPeriod = kronosLogsAll.filter((col('year') >= startSilverDate.year) & (col('month') >= startSilverDate.month)) #& (col('day') >= startSilverDate.day))

In [5]:
from pyspark.sql import functions as F

#Debug
#display(kronosLogsThisPeriod.agg(F.min('day'), F.max('day'),F.min('Month'), F.max('Month'), F.min('Year'), F.max('Year'), F.count("SubmittedTime")))

In [6]:
minDates = kronosLogsThisPeriod.agg({"Year": "min", "Month": "min"}).head()
minMonth = minDates["min(Month)"]
minYear = minDates["min(Year)"]

print("Setting the min values to month =", minMonth, " and Year =", minYear)

Setting the min values to month = 6 and Year = 2020

In [7]:
#debug
print(kronosLogsThisPeriod.count(), " records to to merge")

2529635 records to to merge

In [8]:
#DEBUG - how many items does silver have
#display(kronosLogsThisPeriod.groupBy('Year', 'Month').count())

In [9]:
kronosLogsGroupped = kronosLogsThisPeriod.groupby('year', 'month', 'day', 'FormId', 'LogEventLevel', 'simplifiedMessage', 'StatusCode', 'errorCode').count()

  #display(kronosLogsGroupped.orderBy('year', 'month', 'day', 'formId', 'count'))

In [10]:
#debug
#print(kronosLogsGroupped.count(), " records to to merge")


In [11]:
#DEBUG - how many items there are after the group by sum of counts(I still have the same number of items)
#display(kronosLogsGroupped)

In [12]:
from pyspark.sql.functions import col

#run once to create the initial table structure
#kronosLogsGroupped.write.format('delta').partitionBy('year', 'month').save('/mnt/gold/CAB/Ops/KronosLogs')

In [13]:
kronosLogsGroupped.rdd.getNumPartitions()

Out[13]: 27

In [14]:
kronosLogsGroupped.createOrReplaceTempView('kronosLogsGroupped')

In [15]:
spark.sql(f'''
          MERGE INTO KronosLog as goldKronosLogs
          USING kronosLogsGroupped
          ON goldKronosLogs.year >= {minYear}
              and goldKronosLogs.month >= {minMonth}
              and goldKronosLogs.year = kronosLogsGroupped.year 
              and goldKronosLogs.month = kronosLogsGroupped.month 
              and goldKronosLogs.day = kronosLogsGroupped.day 
              and goldKronosLogs.FormId = kronosLogsGroupped.FormId 
              and goldKronosLogs.LogEventLevel = kronosLogsGroupped.LogEventLevel 
              and goldKronosLogs.simplifiedMessage = kronosLogsGroupped.simplifiedMessage 
              and coalesce(goldKronosLogs.statusCode, '-1') = coalesce(kronosLogsGroupped.statusCode, '-1') 
              and coalesce(goldKronosLogs.errorCode, '-1') = coalesce(kronosLogsGroupped.errorCode, '-1')
          WHEN MATCHED AND goldKronosLogs.count != kronosLogsGroupped.count THEN
            Update set goldKronosLogs.count = kronosLogsGroupped.count
          WHEN NOT MATCHED THEN 
            INSERT *
''')

Out[15]: DataFrame[]

In [16]:
#DEBUG: check the data that was just updated. commented on purpose
kronosLogsGold = spark.read.format('delta').load("/mnt/gold/CAB/Ops/KronosLogs/")
#display(kronosLogsGold.groupBy('Year', 'Month').sum('count'))

#Register the table with Databricks Hive metastore. this only needs to be run once when created to init the table in the hive metastore
#kronosLogsGold.write.saveAsTable("KronosData")
kronosLogsGold.write.format('delta').partitionBy('year').option("path","/mnt/gold/CAB/Ops/KronosLog123/").saveAsTable("KronosLog")  #.option("path","/mnt/gold/CAB/Ops/KronosLog/")

In [17]:
#dbutils.fs.ls('/user/hive/warehouse/kronoslog/year=2020')

In [18]:
spark.catalog.refreshTable("KronosLog")

In [19]:
print("Finished merge gold zone")

Finished merge gold zone